---
##  News Craling 기사 토큰화 하기 

In [90]:
def basicInstalls():
    import subprocess,sys
    # pip가 없으면 pip를 설치한다.
    try:import pip
    except ImportError:
        print("Install pip for python3")
        subprocess.call(['sudo', 'apt-get', 'install', 'python3-pip'])
    try:import selenium        
    except ModuleNotFoundError:
        print("Install Selenium in python3")
        subprocess.call([sys.executable, "-m", "pip", "install", 'selenium'])
    finally:import selenium 

    try:import webdriver_manager
    except ModuleNotFoundError: 
        print("Install webdriver_manager")
        subprocess.call([sys.executable, "-m", "pip", "install", 'webdriver_manager'])
    finally:import webdriver_manager
# basicInstalls()    

def Crawling():
    ### Description ###
    """
        NaverNews Crawling 
        Update:
        2024.06.28 by pdg : basic setting update
            * 기사 하나 긁어서 저장하기 테스트 
    """
    import time, datetime
    import pandas as pd,  numpy as np, matplotlib.pyplot as plt

    ## Beautiful Soup
    from bs4 import BeautifulSoup 

    ## Selenium 
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.support.ui import WebDriverWait
    
    ## Web driver
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.common.by import By
    ## URL
    import requests
    import urllib.request as req
    # Chrome Browser & Chrome Driver Version Check
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')  # 헤드리스 모드 설정 : 웹브라우져가 뒤에서 실행
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options)
    #사이트 열고 기다림 
    time.sleep(np.random.rand()+0.83)
    
    ## 특정 날짜가 없을때  현재 날짜를  세팅
    date = ""
    if date == "":
        today = datetime.date.today()
        date = today.strftime("%Y%m%d")
        print(" 현재 일자 : ",date)
    
    ### URL Setting
    link = f'https://news.naver.com/section/103/&date='
    link = link+date
    ## link 접속
    driver.get(link)
    # driver.implicitly_wait(10000)  # 10초 동안 페이지 로딩 대기
    more_button = WebDriverWait(driver, 2).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'section_more_inner._CONTENT_LIST_LOAD_MORE_BUTTON'))
    )
    itter = 0
    while itter < 2 : # 더보기 5회 
        try :
            itter+=1 
            more_button.click()
            time.sleep(np.random.rand()+0.1)
        except :continue
        
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}

    articles = driver.find_elements(By.CLASS_NAME, 'sa_text_title')
   
    # print(soup)

    # container = soup.select('.sa_item')
    # print(len(container))
    # for i, text in enumerate(container):
    #     lis = text.find_all("li")
    #     # img = text.find_all("img")
    #     try: news_img = text.select_one("img").get("src")
    #     except: news_img = None
    
    
    
    
    print(" -----[ 뉴스 목록 ] ----")
    article_limit = 2
    for idx, article in enumerate(articles):
        if idx < article_limit:
            title = article.text.strip()
            article_link = article.get_attribute('href')
            print(f"{idx+1}.", title)
            print(f"  |- link : {article_link}")
            ##### 뉴스 기사 내용 가져오기 
            response = requests.get(article_link, headers= headers)
            if response.status_code ==200:
                ## Selenium 에서 bs4 로 넘어간다. 
                driver.execute_script("return window.location.href;")
                html = driver.page_source
                soup = BeautifulSoup(html,'html.parser')
                info = soup.find('div',{'id': 'newsct_article'}).text.strip() # 뉴스 기사 
                info = info.replace("\n","")
                print(info)

                
    
    driver.quit()
# Crawling()


def Crawling_ver02(search_word ="", start_page = 1, end_page = 2):
    ### Description ###
    """
        NaverNews Crawling 
        Update:
        2024.06.28 by pdg : basic setting update
            * 기사 하나 긁어서 저장하기 테스트 
    """
    
    ## basic functions 
    import time, datetime, re , sys, random
    import pandas as pd,  numpy as np, matplotlib.pyplot as plt
    from tqdm import tqdm

    ## Beautiful Soup
    from bs4 import BeautifulSoup 

    ## Selenium 
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.support.ui import WebDriverWait
    
    ## Web driver
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.common.by import By
    ## URL
    import requests
    import urllib.request as req
    def makePgNum (num) :
        if num == 1 :
            return num
        elif num == 0 :
            return num + 1
        else :
            return num + 9 * (num - 1)
    # Crawling할 url 생성하는 함수 만들기 (검색어, Crawling 시작 Page, Crawling 종료 Page)
    def makeUrl (search, start_pg, end_pg) :
        if start_pg == end_pg :
            start_page = makePgNum(start_pg)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
            print("생성url: ", url)
            return url
        else :
            urls = []
            for i in range(start_pg, end_pg + 1) :
                page = makePgNum(i)
                url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
                urls.append(url)
            print("생성url: ", urls)
            return urls    
    # html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
    def news_attrs_crawler (articles, attrs) :
        attrs_content = []
        for i in articles :
            attrs_content.append(i.attrs[attrs])
            #***********************************#
            time.sleep(random.uniform(0.3, 0.8))
            #***********************************#
            
        return attrs_content
    # ConnectionError방지
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}
    # html 생성해서 기사 Crawling하는 함수 만들기(url): Link를 반환
    def articles_crawler (url) :
        # html 불러오기
        original_html = requests.get(i, headers = headers)
        #***********************************#
        time.sleep(random.uniform(0.2, 0.8))
        #***********************************#
        html = BeautifulSoup(original_html.text, "html.parser")

        url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
        url = news_attrs_crawler(url_naver, 'href')
        
        return url
    # NAVER url 생성
    url = makeUrl(search_word, start_page, end_page)
    # News Crawler 실행
    news_titles = []
    news_url = []
    news_contents = []
    news_dates = []
    for i in url :
        url = articles_crawler(url)
        news_url.append(url)
    # 제목, Link, 내용 1차원 List로 꺼내는 함수 생성
    def makeList (newlist, content) :
        for i in content :
            for j in i :
                newlist.append(j)
        return newlist
    # 제목, Link, 내용 담을 List 생성
    news_url_1 = []
    # 1차원 List로 만들기 (내용 제외)
    makeList(news_url_1, news_url)
    # NAVER 뉴스만 남기기
    final_urls = []
    for i in tqdm(range(len(news_url_1))) :
        if "news.naver.com" in news_url_1[i] :
            final_urls.append(news_url_1[i])
        else :pass

    ##### News 내용 Crawling #####
    for i in tqdm(final_urls) :
        #각 기사 html get하기
        news = requests.get(i, headers=headers)
        #***********************************#
        time.sleep(random.uniform(0.1, 0.53))
        #***********************************#
        news_html = BeautifulSoup(news.text, "html.parser")


        # News 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None :
            title = news_html.select_one("#content > div.end_ct > div > h2")
        # News 본문 가져오기
        content = news_html.select("article#dic_area")
        if content == [] :
            content = news_html.select("#articeBody")
        # 기사 Text만 가져오기
        # List 합치기
        content = ''.join(str(content))
        # html Tag 제거 및 Text 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern = pattern1, repl = '', string = str(title))
        content = re.sub(pattern = pattern1, repl = '', string = content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')
        news_titles.append(title)
        news_contents.append(content)
        try :
            html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
            news_date = html_date.attrs['data-date-time']
        except AttributeError :
            news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
            news_date = re.sub(pattern = pattern1, repl = '', string = str(news_date))
        # 날짜 가져오기
        news_dates.append(news_date)

    print("검색된 기사 갯수: 총 ", (end_page + 1 - start_page) * 10, '개')
    print("\n[뉴스 제목]")
    print(news_titles)
    print("\n[뉴스 링크]")
    print(final_urls)
    print("\n[뉴스 내용]")
    print(news_contents)
    print()
    print('news_title: ', len(news_titles))
    print('news_url: ', len(final_urls))
    print('news_contents: ', len(news_contents))
    print('news_dates: ', len(news_dates))
    
    
    ##### DataFrame으로 만들기 #####
    import pandas as pd

    # DataFrame 만들기
    news_df = pd.DataFrame(
    {
        'date' : news_dates,
        'title' : news_titles,
        'link' : final_urls,
        'content' : news_contents,
    }
    )

    print(news_df)

    # 중복 Row(행) 지우기
    news_df = news_df.drop_duplicates(keep = 'first', ignore_index = True)
    print("중복 제거 후 행 개수: ", len(news_df))

    # Save to DataFrame
    now = datetime.datetime.now() 
    news_df.to_csv("./NewsData/{}_{}.csv".format(search_word, now.strftime('%Y%m%d_%H시%M분')), encoding = 'utf-8-sig', index=False)
Crawling_ver02(search_word="반도체")

생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=반도체&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=반도체&start=11']


100%|██████████| 20/20 [00:10<00:00,  1.89it/s]

검색된 기사 갯수: 총  20 개

[뉴스 제목]
['한미 반도체 포럼 개최…"반도체 협력이 한미동맹의 핵심"(종합)', '한·미반도체협회, 협력 MOU 체결…공급망 협력 강화', '정부 "바이오는 제2의 반도체"… 특화단지 지정해 키운다', 'SK하이닉스 "AI반도체 리더십 굳힌다"…고성능 SSD 개발', 'SK, 1박2일 전략회의 돌입…"배터리·바이오보다 AI·반도체"', "산업·소비·투자 '트리플 감소'…반도체 선전에도 회복세 둔화(종합)", '한미일 산업장관 첫 회의… 반도체 동맹으로 中 견제 강화', 'TSMC, 日서 2027년 6나노 첨단 반도체 생산', "기업 심리 4개월째 '개선'…반도체·화장품 수출 효과", "디퍼아이, 中·美서 국내 최초 '엣지 AI 반도체' 기술 특허 취득", '한미 표준협력 포럼…"반도체 등 신규 국제표준 공동제안"', '18조원 규모 ‘반도체 금융지원’, 다음달부터 시행', '성남시, 첨단 방위산업용 반도체 설계·검증 지원', '“한미 반도체 중심 AI 협력강화”', '한미일 3국 산업 수장 첫 회의…"반도체-핵심광물 공급망 협력 전선 구축"', '기가레인, 중국 전력반도체시장 진출 본격화', '한미일 산업장관 “반도체·배터리 공급망 강화 협력 최우선”', 'TSMC, 日서 2027년 6나노 첨단 반도체 생산', '삼성 반도체 누리호로 우주 보낸다…나라스페이스와 협력', '반도체·배터리 CAPEX에 고민하는 SK...리밸런싱 방향 잡는다']

[뉴스 링크]
['https://n.news.naver.com/mnews/article/001/0014774976?sid=105', 'https://n.news.naver.com/mnews/article/003/0012634683?sid=101', 'https://n.news.naver.com/mnews/article/417/0001011054?sid=101', 'https://n.news.naver.com/mnews/article/015/0005003246?sid=101', 'ht

---
### 기사 가져오기 

In [95]:
import pandas as pd
df = pd.read_csv("./NewsData/반도체_20240628_17시32분.csv")


In [97]:
df['title']

0           한미 반도체 포럼 개최…"반도체 협력이 한미동맹의 핵심"(종합)
1                 한·미반도체협회, 협력 MOU 체결…공급망 협력 강화
2               정부 "바이오는 제2의 반도체"… 특화단지 지정해 키운다
3             SK하이닉스 "AI반도체 리더십 굳힌다"…고성능 SSD 개발
4           SK, 1박2일 전략회의 돌입…"배터리·바이오보다 AI·반도체"
5         산업·소비·투자 '트리플 감소'…반도체 선전에도 회복세 둔화(종합)
6               한미일 산업장관 첫 회의… 반도체 동맹으로 中 견제 강화
7                  TSMC, 日서 2027년 6나노 첨단 반도체 생산
8                 기업 심리 4개월째 '개선'…반도체·화장품 수출 효과
9         디퍼아이, 中·美서 국내 최초 '엣지 AI 반도체' 기술 특허 취득
10              한미 표준협력 포럼…"반도체 등 신규 국제표준 공동제안"
11                 18조원 규모 ‘반도체 금융지원’, 다음달부터 시행
12                   성남시, 첨단 방위산업용 반도체 설계·검증 지원
13                          “한미 반도체 중심 AI 협력강화”
14    한미일 3국 산업 수장 첫 회의…"반도체-핵심광물 공급망 협력 전선 구축"
15                      기가레인, 중국 전력반도체시장 진출 본격화
16             한미일 산업장관 “반도체·배터리 공급망 강화 협력 최우선”
17                삼성 반도체 누리호로 우주 보낸다…나라스페이스와 협력
18         반도체·배터리 CAPEX에 고민하는 SK...리밸런싱 방향 잡는다
Name: title, dtype: object

In [3]:
import transformers
from transformers import pipeline 
classifier = pipeline("sentiment-analysis") ## text-classification 에러 남

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]